# Azure OpenAI - Function Calling

In [13]:
# Import required packages
import openai
import os
import json
import datetime
import requests

# Define Azure OpenAI endpoint parameters
openai.api_type = "azure"
openai.api_version = "2023-07-01-preview"
    # openai.api_version = "2023-05-15"
    # openai.api_version = "2023-03-15-preview"
openai.api_base = os.getenv("OPENAI_API_BASE") # Set AOAI endpoint value as env variable
openai.api_key = os.getenv("OPENAI_API_KEY") # Set AOAI API key as env variable
aoai_deployment = os.getenv("OPENAI_API_DEPLOY") # Set AOAI deployment name as env variable

# Define Alpha Vantage API parameter
av_api_key = os.getenv("ALPHAVANTAGE_API_KEY") # Set Alpha Vantage API key as env variable

### Testing stock retrieval from Alpha Vantage API

In [2]:
# Get current date
today = datetime.date.today().strftime("%Y-%m-%d")
today

'2023-07-20'

In [60]:
# Function to get stock price from Alpha Vantage API
def get_stock_price(symbol, date):
    print(f"Getting stock price for {symbol} on {date}")
    try:
        stock_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={av_api_key}"
        stock_data = requests.get(stock_url)
        stock_low = stock_data.json()["Time Series (Daily)"][date]["3. low"]
        stock_high = stock_data.json()["Time Series (Daily)"][date]["2. high"]
        return stock_low, stock_high
    except:
        return "NA", "NA"

In [23]:
# Testing stock price function
stock_low, stock_high = get_stock_price("MSFT", today)
print(stock_low, stock_high)

Getting stock price for MSFT on 2023-07-20
345.3700 357.9700


### Describing available functions to GPT model

In [78]:
# Describing required parameters for available functions
functions = [
    {
        "name": "get_stock_price",
        "description": "Retrieve lowest and highest stock price for a given stock symbol and date",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "Stock symbol, for example MSFT for Microsoft"
                },
                "date": {
                    "type": "string",
                    "description": "Date in YYYY-MM-DD format"
                }
            },
            "required": ["symbol", "date"],
        }   
    }
]

available_functions = {
    "get_stock_price": get_stock_price,
} 

### Helper functions
(Adopted from https://github.com/Azure-Samples/openai/tree/main/Basic_Samples/Functions)

In [61]:
import inspect

# Helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [75]:
def run_conversation(messages, functions, available_functions, deployment_id):
    
    # Step 1: Send the conversation and available functions to GPT
    response = openai.ChatCompletion.create(
        deployment_id=aoai_deployment,
        messages=messages,
        functions=functions,
        function_call="auto", 
    )
    response_message = response["choices"][0]["message"]

    # Step 2: Check if GPT model wanted to call a function
    if response_message.get("function_call"):
        print("Recommended function call:")
        print(response_message.get("function_call"))
        print()
        
        # Step 3: call the function

        # Note: the JSON response may not always be valid; be sure to handle errors        
        function_name = response_message["function_call"]["name"]
        
        # Verify that function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name] 
        
        # Verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_args = {k : v for k, v in function_args.items() if v != ""}
        print("Function to call:", function_name)
        print("Function arguments:", function_args)

        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        
        function_response1, function_response2 = function_name(function_args[0], function_args[1])
        function_response = f"Lowest stock price: {function_response1}, Highest stock price: {function_response2}"
        
        print("Output of function call:")
        print(function_response)
        print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # Adding assistant response to messages
        messages.append(
            {
                "role": "assistant",
                "name": function_name,
                "content": response_message["function_call"]["arguments"],
            }
        )

        # Adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        # Complete messages with function response
        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        # Retrieval of second response from GPT
        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=aoai_deployment
        )  
        return second_response

### Practical testing

In [76]:
messages = [{"role": "user", "content": "What is today's stock price for Microsoft shares?"}]
assistant_response = run_conversation(messages, functions, available_functions, aoai_deployment)
#print(assistant_response['choices'][0]['message'])
print(assistant_response)

Recommended function call:
{
  "name": "get_stock_price",
  "arguments": "{\n\"symbol\": \"MSFT\",\n\"date\": \"2022-02-18\"\n}"
}

Function to call: get_stock_price
Function arguments: {'symbol': 'MSFT', 'date': '2022-02-18'}
Invalid number of arguments for function: get_stock_price


In [68]:
check_args(get_stock_price, {"symbol": "MSFT", "date": "2021-08-20"})

True